In [1]:
import numpy as np
import pandas as pd

from sklearn.cluster import MeanShift
from sklearn import preprocessing

import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
%matplotlib inline

In [2]:
df = pd.read_excel('../data/titanic.xls')
original_df = pd.DataFrame.copy(df)

In [3]:
# df.head()

In [4]:
df.drop(['body','name'], 1, inplace=True)
df.convert_objects(convert_numeric=True)
df.fillna(0,inplace=True)

/home/altock/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  from ipykernel import kernelapp as app


In [5]:
def handle_non_numerical_data(df): 
    columns = df.columns.values
    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val]

        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents)

            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x += 1

            df[column] = list(map(convert_to_int,df[column] ))
    
    return df
    
    

In [6]:
df = handle_non_numerical_data(df)

df.drop(['ticket', 'boat'], 1, inplace=True)

In [7]:
X = np.array(df.drop(['survived'], 1).astype(float))
X = preprocessing.scale(X)
y = np.array(df['survived'])

In [20]:
clf = MeanShift()
clf.fit(X)

MeanShift(bandwidth=None, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

In [21]:
labels = clf.labels_
cluster_centers = clf.cluster_centers_

In [22]:
original_df['cluster_group'] = np.nan

for i in range(len(X)):
    original_df['cluster_group'].iloc[i] = labels[i]
    

/home/altock/anaconda3/envs/py35/lib/python3.5/site-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [23]:
survival_rates = {}
n_clusters_ = len(np.unique(labels))

for i in range(n_clusters_):
    temp_df = original_df[(original_df['cluster_group'] == float(i))]
    survival_cluster = temp_df[(temp_df['survived'] == 1)]
    survival_rate = len(survival_cluster)/len(temp_df)
    survival_rates[i] = survival_rate
    
print(survival_rates)

{0: 0.37715179968701096, 1: 0.8095238095238095, 2: 0.1}


In [24]:
print(original_df[(original_df['cluster_group'] == 0)].describe())

            pclass     survived          age        sibsp        parch  \
count  1278.000000  1278.000000  1017.000000  1278.000000  1278.000000   
mean      2.310642     0.377152    29.669289     0.486698     0.321596   
std       0.829032     0.484863    14.393897     1.040027     0.670922   
min       1.000000     0.000000     0.166700     0.000000     0.000000   
25%       2.000000     0.000000          NaN     0.000000     0.000000   
50%       3.000000     0.000000          NaN     0.000000     0.000000   
75%       3.000000     1.000000          NaN     1.000000     0.000000   
max       3.000000     1.000000    80.000000     8.000000     4.000000   

              fare        body  cluster_group  
count  1277.000000  119.000000         1278.0  
mean     28.730948  159.571429            0.0  
std      36.441420   97.302914            0.0  
min       0.000000    1.000000            0.0  
25%            NaN         NaN            0.0  
50%            NaN         NaN            0.0

/home/altock/anaconda3/envs/py35/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)
